#  Entreprise OLIST

SQL
Apprentissage non supervisé
Clustering


## Imports
Nous utiliserons les library standard de datascience : `numpy`, `pandas`, `sklearn`, `matplotlib`. 

In [59]:
import numpy as np
import pandas as pd 
from scipy.stats import randint, uniform

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import *
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.manifold import TSNE

from yellowbrick.cluster import KElbowVisualizer, silhouette_visualizer,SilhouetteVisualizer

import folium

import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')




pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [140]:
def inspect_dupes(df, dedupe=False):
    '''
    Checks duplicates (rows), and gets rid of duplicates if dedupe arg set to 'True' 
    Arg: dataframe, dedupe (bool)
    '''
    num_of_dupe = len(df[df.duplicated()])

    if dedupe and num_of_dupe>0: 
        df.drop_duplicates(inplace=True)
        print(f'Number of duplicates found: {num_of_dupe}')
        return df

    else: 
        print(f'Number of duplicates found: {num_of_dupe}')
        return num_of_dupe
    
def iqr(df,drop=False):
   
    cols_to_keep = [col for col in df.columns if df[col].nunique() > 2]
    dataframe = df[cols_to_keep]
    
    iqr_analysis_list = []

    # Sélection des variables numériques
    numeric_variables = dataframe.select_dtypes(include=['number'])
    l_ind_outlier=[]
    
    for variable_name in numeric_variables.columns:
        # Calcul de l'analyse IQR pour chaque variable numérique
        variable_data = dataframe[variable_name]
        
        # Calcul des quartiles
        Q1 = variable_data.quantile(0.25)
        Q3 = variable_data.quantile(0.75)
        
        # Calcul de l'IQR (Interquartile Range)
        IQR = Q3 - Q1
        
        # Détermination des limites inférieure et supérieure pour détecter les valeurs aberrantes
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Exclusion des individus aberrants
        excluded_count = ((variable_data < lower_bound) | (variable_data > upper_bound)).sum()
        
        # Calcul du ratio d'exclusion
        total_count = len(variable_data)
        exclusion_ratio = excluded_count / total_count
        
        # Ajout des résultats dans une liste
        iqr_analysis_list.append({
            'Variable': variable_name,
            'Q1': Q1,
            'Q3': Q3,
            'IQR': IQR,
            'Lower Bound': lower_bound,
            'Upper Bound': upper_bound,
            'Excluded Count': excluded_count,
            'Exclusion Ratio': exclusion_ratio
        })

        # Filtrer les valeurs en dehors des bornes IQR
        l_ind_outlier += dataframe[(dataframe[variable_name] <= lower_bound) |
                                   (dataframe[variable_name] >= upper_bound)].index.to_list()

    l_ind_outlier=set(l_ind_outlier)
    print(len(l_ind_outlier))
    
    if drop == True :
        dataframe=dataframe.drop(l_ind_outlier,axis=0)
    # Création d'un dataframe à partir de la liste des résultats
    iqr_analysis_df = pd.DataFrame(iqr_analysis_list)
    display(iqr_analysis_df.sort_values(by='Exclusion Ratio',ascending=False))
    
    return dataframe    
    
    
 # Réduit le nombre de décimale
def format_value(x):
    try:
        return f"{x:.2f}"
    except:
        return str(x)
    
  # Explorations de base du df  
def desc(df):
    print( '************************************')  
    print( '      Descriptif du dataset')
    print( '************************************')  
    print(f'Le dataset est composé de {df.shape[0]} individus et {df.shape[1]} variables.')
    display(df.head())
    display(df.dtypes)
    display(df.describe())
    display(bilan_null(df).sort_values(by='Missing ratio',ascending = True))

  # Renvoie un df avec les Nunique et des infos sur les NaN      
def bilan_null(df):
    df_bilan=pd.concat([df.nunique(),
                        df.isnull().mean(),
                        df.isnull().sum(),
                        df.notnull().sum()],
                        axis=1,
                        keys=['Nunique','Missing ratio','Null count','Notnull count'])
    return df_bilan.sort_values(by='Missing ratio')

# Fonction pour encoder les variables catégorielles
def cat_enc(df) :

    le = LabelEncoder()
    le_count = 0

    # Iterate through the columns
    for col in df:
        if df[col].dtype == 'object':
            # If 2 or fewer unique categories
            if len(list(df[col].unique())) <= 2:
                # Train on the training data
                le.fit(df[col])
                # Transform both training and testing data
                df[col] = le.transform(df[col])
                # Keep track of how many columns were label encoded
                le_count += 1

    print('%d columns were label encoded.' % le_count)
    # one-hot encoding of categorical variables
    df = pd.get_dummies(df,dtype=int)

    print('Training Features shape: ', df.shape)
    
    return df
def plot_target_cat(df, cat):

        plt.figure(figsize=(10, 6))
        sns.countplot(x=cat, hue="TARGET", data=df)
        plt.title("Effectifs de {} par {}".format(cat, "TARGET"))
        plt.xlabel(cat)
        if max([len(str(i)) for i in df[cat].unique()]) > 15:
            plt.xticks(rotation=45)
        if max([len(str(i)) for i in df[cat].unique()]) > 20:
            plt.xticks(rotation=90)
        plt.ylabel("Effectifs")
        plt.legend(title="TARGET", loc='upper right')
        plt.show()
        raw_df.groupby(cat)["TARGET"].mean().plot(kind='bar')    
        plt.title("Moyenne de défaut de paiement")
        if max([len(str(i)) for i in df[cat].unique()]) < 15:
            plt.xticks(rotation=0)
        if max([len(str(i)) for i in df[cat].unique()]) > 15:
            plt.xticks(rotation=45)
        if max([len(str(i)) for i in df[cat].unique()]) > 20:
            plt.xticks(rotation=90)
        plt.show()

def plot_target_num(df, numerical_column):

    # Création des bins
    df['bins'] = pd.cut(df[numerical_column], bins=11,precision=0)
    # Calcul de la moyenne de la cible pour chaque bin
    mean_target = df.groupby('bins')["TARGET"].mean()

    plt.figure(figsize=(10, 6))
    sns.countplot(x=df['bins'], hue="TARGET", data=df)
    plt.title("Effectifs de {} par {}".format(numerical_column, "TARGET"))
    plt.xlabel(numerical_column)
    if max([len(str(i)) for i in df[numerical_column].unique()]) > 15:
        plt.xticks(rotation=45)
    if max([len(str(i)) for i in df[numerical_column].unique()]) > 20:
        plt.xticks(rotation=90)
    plt.ylabel("Effectifs")
    plt.legend(title="TARGET", loc='upper right')
    plt.show()    
        
    
    # Tracé du barplot
    plt.figure(figsize=(10, 6))
    mean_target.plot(kind='bar')
    plt.title("Moyenne de défaut de paiement")
    plt.xlabel(numerical_column)
    plt.ylabel(('%'))
    if max([len(str(i)) for i in df[numerical_column].unique()]) > 15:
        plt.xticks(rotation=45)
    if max([len(str(i)) for i in df[numerical_column].unique()]) > 20:
        plt.xticks(rotation=90)
    plt.show()        

def plot_variables(df):
  #  # Plot categorical variables
  #  categorical_cols = df.select_dtypes(include=['object']).columns
  #  for col in categorical_cols:
  #      plt.figure(figsize=(8, 6))
  #      sns.countplot(data=df.sort_values(by=col), x=col)
  #      plt.title(f'{col} Distribution')
  #      plt.xticks(rotation=90)
  #      plt.show()
#
    # Plot numerical variables
    numerical_cols = df.select_dtypes(include=['int', 'float']).columns
    for col in numerical_cols:
        plt.figure(figsize=(8, 6))
        sns.histplot(data=df, x=col, kde=True)
        plt.title(f'{col} Distribution')
        plt.axvline(df[col].mean(), color='red', linestyle='dashed', linewidth=2, label='Mean')
        plt.axvline(df[col].median(), color='green', linestyle='dashed', linewidth=2, label='Median')
        _, p_value = stats.normaltest(df[col].dropna())
        if p_value < 0.05:
            plt.text(1, 0.8, f'p-value: {p_value:.4f}\nNot normally distributed', ha='right', va='top', transform=plt.gca().transAxes, fontsize=10, color='red')
        else:
            plt.text(1, 0.8, f'p-value: {p_value:.4f}\nNormally distributed', ha='right', va='top', transform=plt.gca().transAxes, fontsize=10, color='green')
        plt.legend()
        plt.show()
        
def plot_roc_curves(y_true, d_res):

    plt.figure(figsize=(8, 6))
    for label in d_res :
        print(label)
        print(y_true.shape)
        print(d_res[label][2][:,1].shape)
        if label != 'DummyClassifier' :
            fpr, tpr, _ = roc_curve(y_true, d_res[label][2][:,1])
            auc = roc_auc_score(y_true, d_res[label][2][:,1])
            plt.plot(fpr, tpr, label=f'{label} (AUC = {auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlabel('Taux de faux positifs (FPR)')
    plt.ylabel('Taux de vrais positifs (TPR)')
    plt.title('Courbes ROC')
    plt.legend()
    plt.show()
    
def make_confusion_matrix(y_true,
                          y_pred,
                          y_pred_proba=0,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):
    
    group_names = ['True Neg','False Pos','False Neg','True Pos']

    '''
    This function will make a pretty plot of an sklearn Confusion Matrix cm using a Seaborn heatmap visualization.

    Arguments
    ---------

    categories:    List of strings containing the categories to be displayed on the x,y axis. Default is 'auto'

    count:         If True, show the raw number in the confusion matrix. Default is True.

    cbar:          If True, show the color bar. The cbar values are based off the values in the confusion matrix.
                   Default is True.

    xyticks:       If True, show x and y ticks. Default is True.

    xyplotlabels:  If True, show 'True Label' and 'Predicted Label' on the figure. Default is True.

    sum_stats:     If True, display summary statistics below the figure. Default is True.

    figsize:       Tuple representing the figure size. Default will be the matplotlib rcParams value.

    cmap:          Colormap of the values displayed from matplotlib.pyplot.cm. Default is 'Blues'
                   See http://matplotlib.org/examples/color/colormaps_reference.html
                   
    title:         Title for the heatmap. Default is None.

    '''

    cf=confusion_matrix(y_true, y_pred)

    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks
    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks
    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks
    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])

    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf))

        # if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            # Metrics for Binary Confusion Matrices
            precision = cf[1,1] / np.sum(cf[:,1])
            recall    = cf[1,1] / np.sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            HC_score  = custom_cost(y_true,y_pred)
            auc       = roc_auc_score(y_true, y_pred_proba)
            stats_text = '''
           
HC Score  = {:0.3f}
Accuracy  = {:0.3f}
Precision = {:0.3f}
Recall    = {:0.3f}
F1 Score  = {:0.3f}
                 
ROC AUC   = {:0.3f}
                        '''.format(
                HC_score, accuracy, precision, recall, f1_score, auc)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""


    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)


def search_score(titre, model,  param, n_job, X_train, y_train, X_test, y_test) :
   
    clf = model 
    
    param_grid = param

    model_opt = RandomizedSearchCV(clf,
                                   param_grid,
                                   cv=2,
                                   n_iter=200,
                                   verbose=1,
                                   scoring=HC_scorer,
                                   n_jobs=n_job,
                                   random_state=42,
                                   refit=True )
    
    model_opt = model_opt.fit(X_train, y_train)


    fpr, tpr, thr = roc_curve(y_test, model_opt.predict_proba(X_test)[:,1])
    roc_auc = auc(fpr, tpr)
    
    y_true= y_test
    y_pred = model_opt.predict(X_test)
    y_pred_proba = model_opt.predict_proba(X_test)[:,1]
    
    make_confusion_matrix(y_true, y_pred,  y_pred_proba=y_pred_proba)
    
    d_res[titre]=[
                    roc_auc,
                    custom_cost(y_test,model_opt.predict(X_test)),
                    model_opt.predict_proba(X_test)
                   ]    
   
    results = model_opt.cv_results_

    # Créer un DataFrame avec les informations pertinentes
    df_results = pd.DataFrame({
        'rank_test_score': results['rank_test_score'],
        'mean_test_score': results['mean_test_score'],
        'params': results['params']
    })
    # Extraire les paramètres individuels
    params_df = pd.json_normalize(df_results['params'])

    # Combiner les résultats avec les paramètres
    df_results_combined = pd.concat([df_results.drop(columns=['params']), params_df], axis=1)

    # Trier le DataFrame par rang
    df_results_sorted = df_results_combined.sort_values(by='rank_test_score')


    # Afficher les trois meilleurs résultats
    top_5_results = df_results_sorted.head(5)

    # Afficher les résultats sous forme de tableau
    display(top_5_results)
    
    return model_opt

def correlation_graph(pca,
                      x_y,
                      features) :
    """Affiche le graphe des correlations

    Positional arguments :
    -----------------------------------
    pca : sklearn.decomposition.PCA : notre objet PCA qui a été fit
    x_y : list ou tuple : le couple x,y des plans à afficher, exemple [0,1] pour F1, F2
    features : list ou tuple : la liste des features (ie des dimensions) à représenter
    """

    # Extrait x et y
    x,y=x_y

    # Taille de l'image (en inches)
    fig, ax = plt.subplots(figsize=(10, 9))

    # Pour chaque composante :
    for i in range(0, pca.components_.shape[1]):

        # Les flèches
        ax.arrow(0,0,
                pca.components_[x, i],
                pca.components_[y, i],
                head_width=0.07,
                head_length=0.07,
                width=0.02, )

        # Les labels
        plt.text(pca.components_[x, i] + 0.05,
                pca.components_[y, i] + 0.05,
                features[i])

    # Affichage des lignes horizontales et verticales
    plt.plot([-1, 1], [0, 0], color='grey', ls='--')
    plt.plot([0, 0], [-1, 1], color='grey', ls='--')

    # Nom des axes, avec le pourcentage d'inertie expliqué
    plt.xlabel('F{} ({}%)'.format(x+1, round(100*pca.explained_variance_ratio_[x],1)))
    plt.ylabel('F{} ({}%)'.format(y+1, round(100*pca.explained_variance_ratio_[y],1)))

    # J'ai copié collé le code sans le lire
    plt.title("Cercle des corrélations (F{} et F{})".format(x+1, y+1))

    # Le cercle
    an = np.linspace(0, 2 * np.pi, 100)
    plt.plot(np.cos(an), np.sin(an))  # Add a unit circle for scale

    # Axes et display
    plt.axis('equal')
    plt.show(block=False)

def display_factorial_planes(   X_projected,
                                x_y,
                                pca=None,
                                labels = None,
                                clusters=None,
                                alpha=1,
                                figsize=[10,8],
                                marker="." ):
    """
    Affiche la projection des individus

    Positional arguments :
    -------------------------------------
    X_projected : np.array, pd.DataFrame, list of list : la matrice des points projetés
    x_y : list ou tuple : le couple x,y des plans à afficher, exemple [0,1] pour F1, F2

    Optional arguments :
    -------------------------------------
    pca : sklearn.decomposition.PCA : un objet PCA qui a été fit, cela nous permettra d'afficher la variance de chaque composante, default = None
    labels : list ou tuple : les labels des individus à projeter, default = None
    clusters : list ou tuple : la liste des clusters auquel appartient chaque individu, default = None
    alpha : float in [0,1] : paramètre de transparence, 0=100% transparent, 1=0% transparent, default = 1
    figsize : list ou tuple : couple width, height qui définit la taille de la figure en inches, default = [10,8]
    marker : str : le type de marker utilisé pour représenter les individus, points croix etc etc, default = "."
    """

    # Transforme X_projected en np.array
    X_ = np.array(X_projected)

    # On définit la forme de la figure si elle n'a pas été donnée
    if not figsize:
        figsize = (7,6)

    # On gère les labels
    if  labels is None :
        labels = []
    try :
        len(labels)
    except Exception as e :
        raise e

    # On vérifie la variable axis
    if not len(x_y) ==2 :
        raise AttributeError("2 axes sont demandées")
    if max(x_y )>= X_.shape[1] :
        raise AttributeError("la variable axis n'est pas bonne")

    # on définit x et y
    x, y = x_y

    # Initialisation de la figure
    fig, ax = plt.subplots(1, 1, figsize=figsize)

    # On vérifie s'il y a des clusters ou non
    c = None if clusters is None else clusters

    # Les points
    # plt.scatter(   X_[:, x], X_[:, y], alpha=alpha,
    #                     c=c, cmap="Set1", marker=marker)
    sns.scatterplot(data=None, x=X_[:, x], y=X_[:, y], hue=c)

    # Si la variable pca a été fournie, on peut calculer le % de variance de chaque axe
    if pca :
        v1 = str(round(100*pca.explained_variance_ratio_[x]))  + " %"
        v2 = str(round(100*pca.explained_variance_ratio_[y]))  + " %"
    else :
        v1=v2= ''

    # Nom des axes, avec le pourcentage d'inertie expliqué
    ax.set_xlabel(f'F{x+1} {v1}')
    ax.set_ylabel(f'F{y+1} {v2}')

    # Valeur x max et y max
    x_max = np.abs(X_[:, x]).max() *1.1
    y_max = np.abs(X_[:, y]).max() *1.1

    # On borne x et y
    ax.set_xlim(left=-x_max, right=x_max)
    ax.set_ylim(bottom= -y_max, top=y_max)

    # Affichage des lignes horizontales et verticales
    plt.plot([-x_max, x_max], [0, 0], color='grey', alpha=0.8)
    plt.plot([0,0], [-y_max, y_max], color='grey', alpha=0.8)

    # Affichage des labels des points
    if len(labels) :
        # j'ai copié collé la fonction sans la lire
        for i,(_x,_y) in enumerate(X_[:,[x,y]]):
            plt.text(_x, _y+0.05, labels[i], fontsize='14', ha='center',va='center')

    # Titre et display
    plt.title(f"Projection des individus (sur F{x+1} et F{y+1})")
    plt.show()

##  Chargement des données

df_o = orders_202405281111.csv  
df_i = order_items_202405281111.csv  
df_c = customers_202405281111.csv  
df_r = order_reviews_202405281111.csv  
df_g = extract_groupby.csv  => Résultat de l'aggrégation de feature après l'extraction avec jointure

Requete SQL
> SELECT customer_unique_id,
       COUNT(order_purchase_timestamp) AS COUNT_sale,   
       AVG(review_score) AS AVG_review_score,  
       AVG(price) AS AVG_price,  
       MIN(price) AS MIN_price,  
       MAX(price) AS MAX_price,  
       SUM(price) AS SUM_price,  
       JULIANDAY(DATE('2018-10-17 17:30:18') ) - JULIANDAY(MIN(DATE(order_purchase_timestamp) ) ) AS first_sale,  
       JULIANDAY(DATE('2018-10-17 17:30:18') ) - JULIANDAY(MAX(DATE(order_purchase_timestamp) ) ) AS last_sale,  
       customer_state,  
       review_comment_message,  
       order_status  
  FROM orders o  
       LEFT JOIN  
       customers c ON o.customer_id = c.customer_id  
       LEFT JOIN  
       order_reviews re ON o.order_id = re.order_id  
       LEFT JOIN  
       order_items oi ON o.order_id = oi.order_id  
 GROUP BY customer_unique_id  


Commençons par visualiser 'application_train.csv' : 

In [141]:
# Training data
df_o = pd.read_csv('C:\\Users\Cecil\\Desktop\\Formation\\5.Olist\\data\\orders_202405281111.csv')
df_i = pd.read_csv('C:\\Users\Cecil\\Desktop\\Formation\\5.Olist\\data\\order_items_202405281111.csv')
df_c = pd.read_csv('C:\\Users\Cecil\\Desktop\\Formation\\5.Olist\\data\\customers_202405281111.csv')
df_r = pd.read_csv('C:\\Users\Cecil\\Desktop\\Formation\\5.Olist\\data\\order_reviews_202405281111.csv')
df_g = pd.read_csv('C:\\Users\Cecil\\Desktop\\Formation\\5.Olist\\extract_groupby.csv')

In [142]:
df_o.groupby('order_status').count()

,index,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
order_status,,,,,,,,
approved,2,2,2,2,2,0,0,2
canceled,625,625,625,625,484,75,6,625
created,5,5,5,5,0,0,0,5
delivered,96478,96478,96478,96478,96464,96476,96470,96478
invoiced,314,314,314,314,314,0,0,314
processing,301,301,301,301,301,0,0,301
shipped,1107,1107,1107,1107,1107,1107,0,1107
unavailable,609,609,609,609,609,0,0,609


In [143]:
df_i[df_i['order_id'] == '8272b63d03f5f79c56e9e4120aec44ef']    

,index,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
57297,57297,8272b63d03f5f79c56e9e4120aec44ef,1,270516a3f41dc035aa87d220228f844c,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57298,57298,8272b63d03f5f79c56e9e4120aec44ef,2,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57299,57299,8272b63d03f5f79c56e9e4120aec44ef,3,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57300,57300,8272b63d03f5f79c56e9e4120aec44ef,4,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57301,57301,8272b63d03f5f79c56e9e4120aec44ef,5,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57302,57302,8272b63d03f5f79c56e9e4120aec44ef,6,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57303,57303,8272b63d03f5f79c56e9e4120aec44ef,7,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57304,57304,8272b63d03f5f79c56e9e4120aec44ef,8,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57305,57305,8272b63d03f5f79c56e9e4120aec44ef,9,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57306,57306,8272b63d03f5f79c56e9e4120aec44ef,10,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89


# Analyse exploratoire des données
L'objectif de l'AED est de découvrir ce que nos données peuvent nous apprendre. Elle commence généralement par une vue d'ensemble, puis se concentre sur des domaines spécifiques. Les résultats peuvent par exemple nous aider à décider quelles caractéristiques utiliser pour l'apprentissage du modèle.  

Nous allons commencer par visualiser les informations générales des différents csv :

In [144]:
desc(df_o)

************************************
      Descriptif du dataset
************************************
Le dataset est composé de 99441 individus et 9 variables.


,index,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


index                             int64
order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

,index
count,99441.000000
mean,49720.000000
std,28706.288396
min,0.000000
25%,24860.000000
50%,49720.000000
75%,74580.000000
max,99440.000000


,Nunique,Missing ratio,Null count,Notnull count
index,99441,0.000000,0,99441
order_id,99441,0.000000,0,99441
customer_id,99441,0.000000,0,99441
order_status,8,0.000000,0,99441
order_purchase_timestamp,98875,0.000000,0,99441
order_estimated_delivery_date,459,0.000000,0,99441
order_approved_at,90733,0.001609,160,99281
order_delivered_carrier_date,81018,0.017930,1783,97658
order_delivered_customer_date,95664,0.029817,2965,96476


In [145]:
desc(df_i)

************************************
      Descriptif du dataset
************************************
Le dataset est composé de 112650 individus et 8 variables.


,index,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


index                    int64
order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

,index,order_item_id,price,freight_value
count,112650.000000,112650.000000,112650.000000,112650.000000
mean,56324.500000,1.197834,120.653739,19.990320
std,32519.398249,0.705124,183.633928,15.806405
min,0.000000,1.000000,0.850000,0.000000
25%,28162.250000,1.000000,39.900000,13.080000
50%,56324.500000,1.000000,74.990000,16.260000
75%,84486.750000,1.000000,134.900000,21.150000
max,112649.000000,21.000000,6735.000000,409.680000


,Nunique,Missing ratio,Null count,Notnull count
index,112650,0.0,0,112650
order_id,98666,0.0,0,112650
order_item_id,21,0.0,0,112650
product_id,32951,0.0,0,112650
seller_id,3095,0.0,0,112650
shipping_limit_date,93318,0.0,0,112650
price,5968,0.0,0,112650
freight_value,6999,0.0,0,112650


In [146]:
desc(df_c)

************************************
      Descriptif du dataset
************************************
Le dataset est composé de 99441 individus et 6 variables.


,index,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


index                        int64
customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

,index,customer_zip_code_prefix
count,99441.000000,99441.000000
mean,49720.000000,35137.474583
std,28706.288396,29797.938996
min,0.000000,1003.000000
25%,24860.000000,11347.000000
50%,49720.000000,24416.000000
75%,74580.000000,58900.000000
max,99440.000000,99990.000000


,Nunique,Missing ratio,Null count,Notnull count
index,99441,0.0,0,99441
customer_id,99441,0.0,0,99441
customer_unique_id,96096,0.0,0,99441
customer_zip_code_prefix,14994,0.0,0,99441
customer_city,4119,0.0,0,99441
customer_state,27,0.0,0,99441


In [147]:
desc(df_r)

************************************
      Descriptif du dataset
************************************
Le dataset est composé de 99224 individus et 8 variables.


,index,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


index                       int64
review_id                  object
order_id                   object
review_score                int64
review_comment_title       object
review_comment_message     object
review_creation_date       object
review_answer_timestamp    object
dtype: object

,index,review_score
count,99224.000000,99224.000000
mean,49611.500000,4.086421
std,28643.645892,1.347579
min,0.000000,1.000000
25%,24805.750000,4.000000
50%,49611.500000,5.000000
75%,74417.250000,5.000000
max,99223.000000,5.000000


,Nunique,Missing ratio,Null count,Notnull count
index,99224,0.000000,0,99224
review_id,98410,0.000000,0,99224
order_id,98673,0.000000,0,99224
review_score,5,0.000000,0,99224
review_creation_date,636,0.000000,0,99224
review_answer_timestamp,98248,0.000000,0,99224
review_comment_message,36159,0.587025,58247,40977
review_comment_title,4527,0.883415,87656,11568


A présent nous imputons les valeurs manquantes comme dans le notebook précédent.

In [269]:
desc(df_g)
df_g = df_g[['COUNT_sale','SUM_price','last_sale','AVG_review_score']]
df_g['AVG_review_score'].fillna(df_g['AVG_review_score'].median(), inplace=True)
df_g.fillna(0, inplace=True)

************************************
      Descriptif du dataset
************************************
Le dataset est composé de 96096 individus et 4 variables.


,COUNT_sale,SUM_price,last_sale,AVG_review_score
0,1,129.90,160,5.0
1,1,18.90,163,4.0
2,1,69.00,586,3.0
3,1,25.99,370,4.0
4,1,180.00,337,5.0


COUNT_sale            int64
SUM_price           float64
last_sale             int64
AVG_review_score    float64
dtype: object

,COUNT_sale,SUM_price,last_sale,AVG_review_score
count,96096.000000,95420.000000,96096.000000,95380.000000
mean,1.187271,143.071929,288.108797,4.084694
std,0.650974,218.878967,153.417869,1.342005
min,1.000000,0.850000,0.000000,1.000000
25%,1.000000,47.900000,164.000000,4.000000
50%,1.000000,89.900000,269.000000,5.000000
75%,1.000000,155.990000,397.000000,5.000000
max,24.000000,13440.000000,773.000000,5.000000


,Nunique,Missing ratio,Null count,Notnull count
COUNT_sale,20,0.000000,0,96096
last_sale,632,0.000000,0,96096
SUM_price,8469,0.007035,676,95420
AVG_review_score,60,0.007451,716,95380


Supprimons les commentaires textuels, aucune analyse n'est fourni sur l'analyse sémantique à  ce stade :

In [270]:
df_o.drop('index',axis=1,inplace = True)
df_r.drop('index',axis=1,inplace = True)
df_i.drop('index',axis=1,inplace = True)

KeyError: "['index'] not found in axis"

In [271]:
df_merged1 =       df_o.merge(df_c, on= 'customer_id', how='left')
df_merged2 = df_merged1.merge(df_r, on= 'order_id', how='left')
df_merged3 = df_merged2.merge(df_i, on= 'order_id', how='left')

In [272]:
bilan_null(df_merged1)
bilan_null(df_merged2)
bilan_null(df_merged3)

,Nunique,Missing ratio,Null count,Notnull count
order_id,99441,0.000000,0,114092
customer_city,4119,0.000000,0,114092
customer_zip_code_prefix,14994,0.000000,0,114092
customer_unique_id,96096,0.000000,0,114092
index,99441,0.000000,0,114092
order_estimated_delivery_date,459,0.000000,0,114092
customer_state,27,0.000000,0,114092
order_purchase_timestamp,98875,0.000000,0,114092
order_status,8,0.000000,0,114092
customer_id,99441,0.000000,0,114092


In [274]:
df_merged3['order_purchase_timestamp'] =  pd.to_datetime(df_merged3['order_purchase_timestamp']) 
df_merged3['diff'] =pd.to_datetime('2018-10-17 17:30:18') - df_merged3['order_purchase_timestamp'] 
df_merged3['diff']=df_merged3['diff'].map(lambda x : x.days)

df3_gr = df_merged3.groupby('customer_unique_id', as_index=False, dropna=False).agg( 
    COUNT_sale=('order_purchase_timestamp', 'count'),
    SUM_price=('price', 'sum'),
    last_sale=('diff', 'min'),
    AVG_review_score=('review_score', 'mean')
)

df3_gr['AVG_review_score'].fillna(df3_gr['AVG_review_score'].median(), inplace=True)
df3_gr.fillna(0, inplace=True)

df3_gr.describe().compare(df_g[['COUNT_sale','SUM_price','last_sale','AVG_review_score']].describe())

# (df3_gr[['last_sale']].describe() == df_g[['last_sale']].describe()).all()
# (df3_gr[['COUNT_sale','SUM_price','last_sale']].describe() == df_g[['COUNT_sale','SUM_price','last_sale']].describe()).all()

last_sale            
            self       other
mean  287.735691  288.108797
std   153.414676  153.417869
25%   163.000000  164.000000
50%   268.000000  269.000000
max   772.000000  773.000000

In [277]:
df3_gr.describe()

,COUNT_sale,SUM_price,last_sale,AVG_review_score
count,96096.000000,96096.000000,96096.000000,96096.000000
mean,1.187271,142.065471,287.735691,4.091513
std,0.650974,218.435272,153.414676,1.339311
min,1.000000,0.000000,0.000000,1.000000
25%,1.000000,46.000000,163.000000,4.000000
50%,1.000000,89.000000,268.000000,5.000000
75%,1.000000,154.990000,397.000000,5.000000
max,24.000000,13440.000000,772.000000,5.000000


In [276]:
df_g.describe()

,COUNT_sale,SUM_price,last_sale,AVG_review_score
count,96096.000000,96096.000000,96096.000000,96096.000000
mean,1.187271,142.065471,288.108797,4.091513
std,0.650974,218.435272,153.417869,1.339311
min,1.000000,0.000000,0.000000,1.000000
25%,1.000000,46.000000,164.000000,4.000000
50%,1.000000,89.000000,269.000000,5.000000
75%,1.000000,154.990000,397.000000,5.000000
max,24.000000,13440.000000,773.000000,5.000000


In [255]:
df_g[df_g['SUM_price']<0.001]

,COUNT_sale,SUM_price,last_sale,AVG_review_score


In [246]:
bilan_null(df_g)
df_g.describe()

,COUNT_sale,SUM_price,last_sale,AVG_review_score
count,96096.000000,95420.000000,96096.000000,95380.000000
mean,1.187271,143.071929,288.108797,4.084694
std,0.650974,218.878967,153.417869,1.342005
min,1.000000,0.850000,0.000000,1.000000
25%,1.000000,47.900000,164.000000,4.000000
50%,1.000000,89.900000,269.000000,5.000000
75%,1.000000,155.990000,397.000000,5.000000
max,24.000000,13440.000000,773.000000,5.000000


In [251]:
display(bilan_null(df3_gr))
df3_gr.info()

,Nunique,Missing ratio,Null count,Notnull count
customer_unique_id,96096,0.000000,0,96096
COUNT_sale,20,0.000000,0,96096
SUM_price,8678,0.000000,0,96096
last_sale,630,0.000000,0,96096
AVG_review_score,60,0.007451,716,95380


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96096 entries, 0 to 96095
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_unique_id  96096 non-null  object 
 1   COUNT_sale          96096 non-null  int64  
 2   SUM_price           96096 non-null  float64
 3   last_sale           96096 non-null  int64  
 4   AVG_review_score    95380 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 3.7+ MB


In [250]:
display(bilan_null(df_g))
df_g.info()

,Nunique,Missing ratio,Null count,Notnull count
COUNT_sale,20,0.000000,0,96096
last_sale,632,0.000000,0,96096
SUM_price,8469,0.007035,676,95420
AVG_review_score,60,0.007451,716,95380


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96096 entries, 0 to 96095
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   COUNT_sale        96096 non-null  int64  
 1   SUM_price         95420 non-null  float64
 2   last_sale         96096 non-null  int64  
 3   AVG_review_score  95380 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 2.9 MB


In [22]:
# sample data
df = pd.DataFrame(pd.date_range('2018-01-01', '2018-03-01'), columns=['Date'])
# groupby your key and freq
g = df.groupby(pd.Grouper(key='Date', freq='M'))
# groups to a list of dataframes with list comprehension
dfs = [group for _,group in g]

,customer_unique_id,COUNT_sale,AVG_review_score,AVG_price,MIN_price,MAX_price,SUM_price,first_sale,last_sale,customer_state,order_status
0,0000366f3b9a7992bf8c76cfdf3221e2,1,5.0,129.90,129.90,129.90,129.90,160,160,SP,delivered
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,4.0,18.90,18.90,18.90,18.90,163,163,SP,delivered
2,0000f46a3911fa3c0805444483337064,1,3.0,69.00,69.00,69.00,69.00,586,586,SC,delivered
3,0000f6ccb0745a6a4b88665a16c9f078,1,4.0,25.99,25.99,25.99,25.99,370,370,PA,delivered
4,0004aac84e0df4da2b147fca70cf8255,1,5.0,180.00,180.00,180.00,180.00,337,337,SP,delivered
...,...,...,...,...,...,...,...,...,...,...,...
96091,fffcf5a5ff07b0908bd4e2dbc735a684,2,5.0,785.00,680.00,890.00,1570.00,496,496,PE,delivered
96092,fffea47cd6d3cc0a88bd621562a9d061,1,4.0,64.89,64.89,64.89,64.89,311,311,BA,delivered
96093,ffff371b4d645b6ecea244b27531430a,1,5.0,89.90,89.90,89.90,89.90,617,617,MT,delivered
96094,ffff5962728ec6157033ef9805bacc48,1,5.0,115.00,115.00,115.00,115.00,168,168,ES,delivered
